Etude dataset

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import pyarrow.parquet as pq
import torch.nn as nn

In [3]:
df = pd.read_parquet('/Users/loufourneaux/Desktop/EPFL/MA1/ML/project2/All_Relative_Results_Cleaned.parquet', engine='pyarrow')

In [5]:
df


,Participant,Exercise,Set,Camera,time(s),left_ankle_x,left_ankle_y,left_ankle_z,left_ear_x,left_ear_y,...,right_pinky_z,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_thumb_x,right_thumb_y,right_thumb_z,right_wrist_x,right_wrist_y,right_wrist_z
0,P04,Abduction,Correct,Frontal_Top,0.000000,0.009328,0.779305,0.187880,0.020658,-0.693419,...,-0.633111,-0.054347,-0.511760,-0.185118,-0.068159,-0.602015,-0.598093,-0.066565,-0.600690,-0.592768
1,P04,Abduction,Correct,Frontal_Top,0.033333,0.009353,0.779298,0.185050,0.021187,-0.693361,...,-0.630835,-0.054293,-0.511703,-0.184861,-0.068130,-0.602200,-0.596219,-0.066508,-0.600835,-0.590916
2,P04,Abduction,Correct,Frontal_Top,0.066667,0.009722,0.779311,0.179212,0.021351,-0.693388,...,-0.630923,-0.054289,-0.511696,-0.184849,-0.067241,-0.601852,-0.596222,-0.065676,-0.600436,-0.590910
3,P04,Abduction,Correct,Frontal_Top,0.100000,0.011696,0.778027,0.166566,0.023431,-0.693102,...,-0.631444,-0.053769,-0.511612,-0.183764,-0.062897,-0.600404,-0.596646,-0.061278,-0.599290,-0.591304
4,P04,Abduction,Correct,Frontal_Top,0.133333,0.011671,0.778045,0.166878,0.023658,-0.693346,...,-0.631149,-0.053763,-0.512331,-0.183866,-0.061787,-0.600463,-0.596128,-0.060647,-0.599425,-0.590991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,P28,0,E,Side_Low,24.333333,0.104921,0.764555,0.082647,0.058097,-0.669222,...,-0.153244,-0.163026,-0.512648,-0.044837,-0.626770,-0.533090,-0.155961,-0.617913,-0.532957,-0.135954
731,P28,0,E,Side_Low,24.366667,0.104855,0.764961,0.084220,0.058165,-0.668928,...,-0.157576,-0.163073,-0.512568,-0.046608,-0.626767,-0.531939,-0.159559,-0.617969,-0.531721,-0.139373
732,P28,0,E,Side_Low,24.400000,0.105392,0.765160,0.084043,0.058210,-0.668465,...,-0.158064,-0.163069,-0.512303,-0.046589,-0.626679,-0.530991,-0.159817,-0.617878,-0.530680,-0.139622
733,P28,0,E,Side_Low,24.433333,0.106408,0.765264,0.084009,0.059145,-0.668389,...,-0.158204,-0.162886,-0.512327,-0.046626,-0.625844,-0.529880,-0.159893,-0.617153,-0.529503,-0.139712


#steps:
1) Clean dataset
2) Split dataset: training+validation+test
3) Build CNN architecture
4) Compile the model using appropriate classification and loss function
5) Train the model and adjust hyperparameters 
6) Evaluated model and optimize 

###Load data 

In [13]:
###ouvrir dataset en utlisant torch

class JointPointsDataset(Dataset):
    def __init__(self, parquet_file):
        self.data = pq.read_table(parquet_file).to_pandas()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Obtenez les différentes informations de la ligne idx
        participant_id = self.data.iloc[idx, 1]
        exercise_type = self.data.iloc[idx, 2]
        camera_type = self.data.iloc[idx, 3]
        time = self.data.iloc[idx, 4]

        # Obtenez les coordonnées des joints du corps (à partir de la 6e colonne)
        joint_points = self.data.iloc[idx, 5:].values.astype(float)

        # Créez un dictionnaire avec toutes les informations
        sample = {
            'participant_id': participant_id,
            'exercise_type': exercise_type,
            'camera_type': camera_type,
            'time': time,
            'joint_points': joint_points
        }

        # Appliquez des transformations si nécessaire

        return sample



In [ ]:


# Créer une instance de votre dataset avec un fichier Parquet
dataset = JointPointsDataset(parquet_file='/Users/loufourneaux/Desktop/EPFL/MA1/ML/project2/All_Relative_Results_Cleaned.parquet')

# Créer un DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Utiliser le DataLoader pour itérer sur vos données pendant l'entraînement
for batch in dataloader:
    participant_ids = batch['participant_id']
    joint_points = batch['joint_points']


###construire un model

In [3]:
import torch.nn.functional as F

class Model(nn.Module):
    
    #input layer of network--> hidden layer 1--> hidden layer---> output layer (7 classes)
    def __init__(self, input_features=99, h1=8, h2=8, output_features=7):
        super().__init__()
        #moving from one layer to another
        self.fc1=nn.Linear(input_features,h1)
        self.fc2=nn.Linear(h1, h2)
        self.out=nn.Linear(h2, output_features)

    def forward(self, x): #need to actually move from layer to layer 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)

        return x


TRAIN MODEL

In [4]:
#change exercice column from strings to interger
df['Exercise']= df['Exercise'].replace('Stretch', 0)
#... do this fro all of them for classification

In [4]:
#train test split X, y
X = df.drop(columns=['Participant','Exercise', 'Set', 'Camera'], axis=1)
y= df['Exercise']

#convert to numpy arrays
X= X.values
y=y.values

In [2]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

In [1]:
#want data to be tensors 
X_train=torch.FloatTensor(X_train)
X_test=torch.FloatTensor(X_test)
y_test=torch.FloatTensor(y_test)
y_train=torch.FloatTensor(y_train)


NameError: name 'torch' is not defined

In [ ]:
#set criterion of model to see how far off the predictions are 
criterion=nn.CrossEntropyLoss()
#choose optimizer, set learning rate lr (if error doesn't go down after iteration epochs, want to lower learning rate)make lr lower=> take more time
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

#1 epoch = 1 iteration on model 
epochs=100
losses=[]

for i in range(epochs):
    #go forward and get prediction 
    y_pred= model.forward(X_train) #get predicted result

    #measure loss
    loss=criterion(y_pred, y_train)#predicted value vs y train
    losses.append(loss.detach().numpy())#keep track of losses as np

    if i%10==0:
        print(f'Epoch: {i} loss:{loss}')

    #do back propagation: take error rate and feed it back through the network to fine tune the weights 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    

